# 第二课 词向量

第二课学习目标
- 学习词向量的概念
- 用Skip-thought模型训练词向量
- 学习使用PyTorch dataset和dataloader
- 学习定义PyTorch模型
- 学习torch.nn中常见的Module
    - Embedding
- 学习常见的PyTorch operations
    - bmm
    - logsigmoid
- 保存和读取PyTorch模型
    

第二课使用的训练数据可以从以下链接下载到。

链接:https://pan.baidu.com/s/1tFeK3mXuVXEy3EMarfeWvg  密码:v2z5

在这一份notebook中，我们会（尽可能）尝试复现论文[Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)中训练词向量的方法. 我们会实现Skip-gram模型，并且使用论文中noice contrastive sampling的目标函数。

这篇论文有很多模型实现的细节，这些细节对于词向量的好坏至关重要。我们虽然无法完全复现论文中的实验结果，主要是由于计算资源等各种细节原因，但是我们还是可以大致展示如何训练词向量。

以下是一些我们没有实现的细节
- subsampling：参考论文section 2.3

In [15]:
import torch
import torch.nn as nn  #神经网络工具箱torch.nn 
import torch.nn.functional as F  #神经网络函数torch.nn.functional
import torch.utils.data as tud  #Pytorch读取训练集需要用到torch.utils.data类

**两个模块的区别：**[torch.nn 和 torch.functional 的区别](https://blog.csdn.net/hawkcici160/article/details/80140059)

In [16]:
from torch.nn.parameter import Parameter  #参数更新和优化函数

from collections import Counter #Counter 计数器
import numpy as np 
import random
import math 

import pandas as pd
import scipy #SciPy是基于NumPy开发的高级模块，它提供了许多数学算法和函数的实现
import sklearn
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度函数

In [17]:
USE_CUDA = torch.cuda.is_available() #有GPU可以用

# 为了保证实验结果可以复现，我们经常会把各种random seed固定在某一个值
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)
    
# 设定一些超参数   
K = 100 # number of negative samples 负样本随机采样数量
C = 3 # nearby words threshold 指定周围三个单词进行预测
NUM_EPOCHS = 2 # The number of epochs of training 迭代轮数
MAX_VOCAB_SIZE = 30000 # the vocabulary size 词汇表多大
BATCH_SIZE = 128 # the batch size 每轮迭代1个batch的数量
LEARNING_RATE = 0.2 # the initial learning rate #学习率
EMBEDDING_SIZE = 100 #词向量维度
       
    
LOG_FILE = "word-embedding.log"

# tokenize函数，把一篇文本转化成一个个单词
def word_tokenize(text): 
    return text.split()

- 从文本文件中读取所有的文字，通过这些文本创建一个vocabulary
- 由于单词数量可能太大，我们只选取最常见的MAX_VOCAB_SIZE个单词
- 我们添加一个UNK单词表示所有不常见的单词
- 我们需要记录单词到index的mapping，以及index到单词的mapping，单词的count，单词的(normalized) frequency，以及单词总数。

In [18]:
with open("text8.train.txt", "r") as fin: #读入文件
    text = fin.read()
    
text = [w for w in word_tokenize(text.lower())] 
#分词，在这里类似于text.split()

vocab = dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))
#字典格式，把（MAX_VOCAB_SIZE-1）个最频繁出现的单词取出来，-1是留给不常见的单词

vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))
#unk表示不常见单词数=总单词数-常见单词数
#这里计算的到vocab["<unk>"]=29999

idx_to_word = [word for word in vocab.keys()] 
#取出字典的所有单词key

word_to_idx = {word:i for i, word in enumerate(idx_to_word)}
#取出所有单词的单词和对应的索引，索引值与单词出现次数相反，最常见单词索引为0。

word_counts = np.array([count for count in vocab.values()], dtype=np.float32)
#所有单词的频数values

word_freqs = word_counts / np.sum(word_counts)
#所有单词的频率

word_freqs = word_freqs ** (3./4.)
#论文里乘以3/4次方

word_freqs = word_freqs / np.sum(word_freqs) # 用来做 negative sampling
# 重新计算所有单词的频率

VOCAB_SIZE = len(idx_to_word) #词汇表单词数30000=MAX_VOCAB_SIZE
VOCAB_SIZE

30000

### 实现Dataloader

一个dataloader需要以下内容：

- 把所有text编码成数字，然后用subsampling预处理这些文字。
- 保存vocabulary，单词count，normalized word frequency
- 每个iteration sample一个中心词
- 根据当前的中心词返回context单词
- 根据中心词sample一些negative单词
- 返回单词的counts

这里有一个好的tutorial介绍如何使用[PyTorch dataloader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).
为了使用dataloader，我们需要定义以下两个function:

- ```__len__``` function需要返回整个数据集中有多少个item
- ```__get__``` 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。

In [19]:
class WordEmbeddingDataset(tud.Dataset): #tud.Dataset父类
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        ''' text: a list of words, all text from the training dataset
            word_to_idx: the dictionary from word to idx
            idx_to_word: idx to word mapping
            word_freq: the frequency of each word
            word_counts: the word counts
        '''
        super(WordEmbeddingDataset, self).__init__() #初始化模型
        self.text_encoded = [word_to_idx.get(t, VOCAB_SIZE-1) for t in text]
        #字典 get() 函数返回指定键的值（第一个参数），如果值不在字典中返回默认值（第二个参数）。
        #取出text里每个单词word_to_idx字典里对应的索引,不在字典里返回"<unk>"的索引
        #"<unk>"的索引=29999，get括号里第二个参数应该写word_to_idx["<unk>"]，不应该写VOCAB_SIZE-1，虽然数值一样。
        
        self.text_encoded = torch.Tensor(self.text_encoded).long()
        #变成tensor类型，这里变成longtensor，也可以torch.LongTensor(self.text_encoded)
        
        self.word_to_idx = word_to_idx #保存数据
        self.idx_to_word = idx_to_word  #保存数据
        self.word_freqs = torch.Tensor(word_freqs) #保存数据
        self.word_counts = torch.Tensor(word_counts) #保存数据
        
    def __len__(self): #数据集有多少个item 
        #魔法函数__len__
        ''' 返回整个数据集（所有单词）的长度
        '''
        return len(self.text_encoded) #所有单词的总数
        
    def __getitem__(self, idx):
        #魔法函数__getitem__，这个函数跟普通函数不一样
        ''' 这个function返回以下数据用于训练
            - 中心词
            - 这个单词附近的(positive)单词
            - 随机采样的K个单词作为negative sample
        '''
        center_word = self.text_encoded[idx] 
        #print(center_word)
        #中心词索引
        #这里__getitem__函数是个迭代器，idx代表了所有的单词索引。
        
        pos_indices = list(range(idx-C, idx)) + list(range(idx+1, idx+C+1))
        #周围词索引的索引，比如idx=0时。pos_indices = [-3, -2, -1, 1, 2, 3] 
        #老师讲这里的时候，我不是特别明白
        
        pos_indices = [i%len(self.text_encoded) for i in pos_indices]
        #range(idx+1, idx+C+1)超出词汇总数时，需要特别处理，取余数
        
        pos_words = self.text_encoded[pos_indices]
        #周围词索引，就是希望出现的正例单词
        #print(pos_words)
        
        neg_words = torch.multinomial(self.word_freqs, K * pos_words.shape[0], True)
        #负例采样单词索引，torch.multinomial作用是对self.word_freqs做K * pos_words.shape[0]次取值，输出的是self.word_freqs对应的下标。
        #取样方式采用有放回的采样，并且self.word_freqs数值越大，取样概率越大。
        #每个正确的单词采样K个，pos_words.shape[0]是正确单词数量
        #print(neg_words)
        
        return center_word, pos_words, neg_words 

创建dataset和dataloader

In [20]:
dataset = WordEmbeddingDataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)
# list(dataset) 可以把尝试打印下center_word, pos_words, neg_words看看

dataloader = tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)     

torch.utils.data.DataLoader理解：https://blog.csdn.net/qq_36653505/article/details/83351808

### 定义PyTorch模型

In [21]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        ''' 初始化输出和输出embedding
        '''
        super(EmbeddingModel, self).__init__()
        self.vocab_size = vocab_size  #30000
        self.embed_size = embed_size  #100
        
        initrange = 0.5 / self.embed_size
        self.out_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        #模型输出nn.Embedding(30000, 100)
        self.out_embed.weight.data.uniform_(-initrange, initrange)
        #权重初始化的一种方法
        
        
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
         #模型输入nn.Embedding(30000, 100)
        self.in_embed.weight.data.uniform_(-initrange, initrange)
        #权重初始化的一种方法
        
        
    def forward(self, input_labels, pos_labels, neg_labels):
        '''
        input_labels: 中心词, [batch_size]
        pos_labels: 中心词周围 context window 出现过的单词 [batch_size * (window_size * 2)]
        neg_labelss: 中心词周围没有出现过的单词，从 negative sampling 得到 [batch_size, (window_size * 2 * K)]
        
        return: loss, [batch_size]
        '''
        
        batch_size = input_labels.size(0)  #input_labels是输入的标签，tud.DataLoader()返回的。相已经被分成batch了。
        
        input_embedding = self.in_embed(input_labels) 
        # B * embed_size
        #这里估计进行了运算：（128,30000）*（30000,100）= 128(B) * 100 (embed_size)
        
        pos_embedding = self.out_embed(pos_labels) # B * (2*C) * embed_size
        #同上，增加了维度(2*C)，表示一个batch有B组周围词单词，一组周围词有(2*C)个单词，每个单词有embed_size个维度。
        
        neg_embedding = self.out_embed(neg_labels) # B * (2*C * K) * embed_size
        #同上，增加了维度(2*C*K)
      
    
        #torch.bmm()为batch间的矩阵相乘（b,n.m)*(b,m,p)=(b,n,p)
        log_pos = torch.bmm(pos_embedding, input_embedding.unsqueeze(2)).squeeze() # B * (2*C)
        log_neg = torch.bmm(neg_embedding, -input_embedding.unsqueeze(2)).squeeze() # B * (2*C*K)
        #unsqueeze(2)指定位置升维，.squeeze()压缩维度。
        
        #下面loss计算就是论文里的公式
        log_pos = F.logsigmoid(log_pos).sum(1)
        log_neg = F.logsigmoid(log_neg).sum(1) # batch_size     
        loss = log_pos + log_neg
        
        return -loss
    
    def input_embeddings(self):   #取出self.in_embed数据参数
        return self.in_embed.weight.data.cpu().numpy()
        

定义一个模型以及把模型移动到GPU

In [22]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_SIZE)
#得到model，有参数，有loss，可以优化了

if USE_CUDA:
    model = model.cuda()

下面是评估模型的代码，以及训练模型的代码

In [23]:
def evaluate(filename, embedding_weights): 
    if filename.endswith(".csv"):
        data = pd.read_csv(filename, sep=",")
    else:
        data = pd.read_csv(filename, sep="\t")
    human_similarity = []
    model_similarity = []
    for i in data.iloc[:, 0:2].index:
        word1, word2 = data.iloc[i, 0], data.iloc[i, 1]
        if word1 not in word_to_idx or word2 not in word_to_idx:
            continue
        else:
            word1_idx, word2_idx = word_to_idx[word1], word_to_idx[word2]
            word1_embed, word2_embed = embedding_weights[[word1_idx]], embedding_weights[[word2_idx]]
            model_similarity.append(float(sklearn.metrics.pairwise.cosine_similarity(word1_embed, word2_embed)))
            human_similarity.append(float(data.iloc[i, 2]))

    return scipy.stats.spearmanr(human_similarity, model_similarity)# , model_similarity

def find_nearest(word):
    index = word_to_idx[word]
    embedding = embedding_weights[index]
    cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]]

训练模型：
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass，通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，以及在验证数据集上做模型的评估

In [24]:
for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):
        print(input_labels, pos_labels, neg_labels)
        if i>5:
            break


tensor([ 2070,     4,   575,    43,   310,    19,     3,    10,    19,  2898,
           37,    18,  2996,    31,   645,    19,    13,  3533,   146,     0,
         2402,    81,  1863, 10821,     7,   497,  2406,     1,    81,    75,
            3,  8046,    51,  8914,   413,     0,  3369,   884,     0,    29,
          732, 13046,   213,     4,   164,    19, 23668, 12285,  8111,  2207,
          261,     3,   470,   134, 10383,     6,   310,     0,   310,    29,
          415,    72,  2174,     1,   123,  7483,   110,   577,   177,     0,
            0,    30,     6,    27,    26,   846,    39, 29999,  1776,    46,
            3,     5,     4,    11,     7, 16394,  2087,  1410,     9,    20,
        29999, 14598, 11477,   799,   104,     3,  3328,     1,    44,    30,
           14,     9, 29999,  2778,     3,     0,    41, 21367,    11,     1,
           10,    33,    12,  4688,  5279,     3,  3894, 29999, 29999,   209,
         1204,    71,   596,  2303,  9274,    15, 29999,    11])

tensor([ 8480, 11692,  3942,     5,   372,    19,   759, 29999,     0,  4735,
          646, 29999,    62,  1302,  8695,   119,    18,    15,     9,   584,
           65,     8,     5,     0,     6,   966,   241,  7036,    14,    40,
           10,    55,    10,    14,     1,   149, 29999,    28,  4938,     3,
           32,    76,  9187,   313,    72,     0,     5,     0,   736,     3,
            9,    52, 22471,  1009,     5,  4760,    44,    10,  3385,  3864,
         9483,    10,     2,     7,  1012,   183, 12791,     9,  5854,  8422,
         4459,     1,   472,     6,    81,    13,    21,     7,    25,    10,
            3,   867,   130, 25963,    48, 21883, 15664,     2,   164,  1091,
        13747,   556,  2672,     2,    13,  3437, 23778,     8,   104,    40,
          946,    21, 12009,   210,    41,    13,   356,  5337,     0,    85,
            6,     1, 18467,     8,     0,  1364,     4, 23112,    31,     4,
            0,    19,     5,     4,  1540, 15862,    11,     6])

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
#随机梯度下降

for e in range(NUM_EPOCHS): #开始迭代
    for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):
        #print(input_labels, pos_labels, neg_labels)
        
        # TODO
        input_labels = input_labels.long() #longtensor
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()
       
        #下面第一节课都讲过的   
        optimizer.zero_grad() #梯度归零
        loss = model(input_labels, pos_labels, neg_labels).mean()
        
        loss.backward()
        optimizer.step()
       
        #打印结果。
        if i % 100 == 0:
            with open(LOG_FILE, "a") as fout:
                fout.write("epoch: {}, iter: {}, loss: {}\n".format(e, i, loss.item()))
                print("epoch: {}, iter: {}, loss: {}".format(e, i, loss.item()))
            
        
        if i % 2000 == 0:
            embedding_weights = model.input_embeddings()
            sim_simlex = evaluate("simlex-999.txt", embedding_weights)
            sim_men = evaluate("men.txt", embedding_weights)
            sim_353 = evaluate("wordsim353.csv", embedding_weights)
            with open(LOG_FILE, "a") as fout:
                print("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                fout.write("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                
    embedding_weights = model.input_embeddings()
    np.save("embedding-{}".format(EMBEDDING_SIZE), embedding_weights)
    torch.save(model.state_dict(), "embedding-{}.th".format(EMBEDDING_SIZE))

epoch: 0, iter: 0, loss: 420.04736328125
epoch: 0, iteration: 0, simlex-999: SpearmanrResult(correlation=0.002806243285464091, pvalue=0.9309107582703205), men: SpearmanrResult(correlation=-0.03578915454199749, pvalue=0.06854012381329619), sim353: SpearmanrResult(correlation=0.02468906830123471, pvalue=0.6609497549092586), nearest to monster: ['monster', 'communism', 'bosses', 'microprocessors', 'infectious', 'debussy', 'unesco', 'tantamount', 'offices', 'tischendorf']

epoch: 0, iter: 100, loss: 278.9967041015625
epoch: 0, iter: 200, loss: 248.71990966796875
epoch: 0, iter: 300, loss: 202.95816040039062
epoch: 0, iter: 400, loss: 157.04776000976562
epoch: 0, iter: 500, loss: 137.83531188964844
epoch: 0, iter: 600, loss: 121.03585815429688
epoch: 0, iter: 700, loss: 105.300537109375
epoch: 0, iter: 800, loss: 114.10055541992188
epoch: 0, iter: 900, loss: 104.72723388671875
epoch: 0, iter: 1000, loss: 99.03569030761719
epoch: 0, iter: 1100, loss: 95.2179946899414
epoch: 0, iter: 1200, lo

epoch: 0, iter: 12100, loss: 33.59938430786133
epoch: 0, iter: 12200, loss: 32.594879150390625
epoch: 0, iter: 12300, loss: 32.42393493652344
epoch: 0, iter: 12400, loss: 32.8863410949707
epoch: 0, iter: 12500, loss: 39.303016662597656
epoch: 0, iter: 12600, loss: 33.103118896484375
epoch: 0, iter: 12700, loss: 36.31195068359375
epoch: 0, iter: 12800, loss: 33.8329963684082
epoch: 0, iter: 12900, loss: 32.499595642089844
epoch: 0, iter: 13000, loss: 33.224632263183594
epoch: 0, iter: 13100, loss: 33.931884765625
epoch: 0, iter: 13200, loss: 33.35892105102539
epoch: 0, iter: 13300, loss: 33.33966064453125
epoch: 0, iter: 13400, loss: 34.09075164794922
epoch: 0, iter: 13500, loss: 33.52397918701172
epoch: 0, iter: 13600, loss: 34.18444061279297
epoch: 0, iter: 13700, loss: 33.96720886230469
epoch: 0, iter: 13800, loss: 34.23271942138672
epoch: 0, iter: 13900, loss: 33.36094665527344
epoch: 0, iter: 14000, loss: 35.998287200927734
epoch: 0, iteration: 14000, simlex-999: SpearmanrResult(co

epoch: 0, iter: 24200, loss: 31.729236602783203
epoch: 0, iter: 24300, loss: 31.751216888427734
epoch: 0, iter: 24400, loss: 31.54802131652832
epoch: 0, iter: 24500, loss: 31.819448471069336
epoch: 0, iter: 24600, loss: 31.87582778930664
epoch: 0, iter: 24700, loss: 32.44230651855469
epoch: 0, iter: 24800, loss: 32.13909149169922
epoch: 0, iter: 24900, loss: 31.6838321685791
epoch: 0, iter: 25000, loss: 32.01523208618164
epoch: 0, iter: 25100, loss: 31.727489471435547
epoch: 0, iter: 25200, loss: 32.378543853759766
epoch: 0, iter: 25300, loss: 32.155052185058594
epoch: 0, iter: 25400, loss: 32.30049514770508
epoch: 0, iter: 25500, loss: 32.10628128051758
epoch: 0, iter: 25600, loss: 32.01287841796875
epoch: 0, iter: 25700, loss: 32.22496032714844
epoch: 0, iter: 25800, loss: 32.15202331542969
epoch: 0, iter: 25900, loss: 32.43567657470703
epoch: 0, iter: 26000, loss: 31.745975494384766
epoch: 0, iteration: 26000, simlex-999: SpearmanrResult(correlation=0.08715629365703002, pvalue=0.006

epoch: 0, iter: 36400, loss: 31.05801773071289
epoch: 0, iter: 36500, loss: 31.969802856445312
epoch: 0, iter: 36600, loss: 31.290489196777344
epoch: 0, iter: 36700, loss: 31.409465789794922
epoch: 0, iter: 36800, loss: 31.444076538085938
epoch: 0, iter: 36900, loss: 31.494474411010742
epoch: 0, iter: 37000, loss: 31.12554931640625
epoch: 0, iter: 37100, loss: 31.744049072265625
epoch: 0, iter: 37200, loss: 31.608917236328125
epoch: 0, iter: 37300, loss: 31.441722869873047
epoch: 0, iter: 37400, loss: 31.544227600097656
epoch: 0, iter: 37500, loss: 31.359806060791016
epoch: 0, iter: 37600, loss: 31.130847930908203
epoch: 0, iter: 37700, loss: 32.14916229248047
epoch: 0, iter: 37800, loss: 31.148212432861328
epoch: 0, iter: 37900, loss: 31.835248947143555
epoch: 0, iter: 38000, loss: 31.421974182128906
epoch: 0, iteration: 38000, simlex-999: SpearmanrResult(correlation=0.09401565185194706, pvalue=0.003602024110356835), men: SpearmanrResult(correlation=0.09723017395213002, pvalue=7.10171

epoch: 0, iter: 48500, loss: 31.48914909362793
epoch: 0, iter: 48600, loss: 31.45376205444336
epoch: 0, iter: 48700, loss: 30.948339462280273
epoch: 0, iter: 48800, loss: 30.842824935913086
epoch: 0, iter: 48900, loss: 30.931697845458984
epoch: 0, iter: 49000, loss: 31.468204498291016
epoch: 0, iter: 49100, loss: 31.04726791381836
epoch: 0, iter: 49200, loss: 31.148698806762695
epoch: 0, iter: 49300, loss: 31.295198440551758
epoch: 0, iter: 49400, loss: 31.415983200073242
epoch: 0, iter: 49500, loss: 31.53121566772461
epoch: 0, iter: 49600, loss: 30.391773223876953
epoch: 0, iter: 49700, loss: 31.365924835205078
epoch: 0, iter: 49800, loss: 30.920448303222656
epoch: 0, iter: 49900, loss: 30.881540298461914
epoch: 0, iter: 50000, loss: 31.272510528564453
epoch: 0, iteration: 50000, simlex-999: SpearmanrResult(correlation=0.10413335271622073, pvalue=0.0012554545146236879), men: SpearmanrResult(correlation=0.10361287469529604, pvalue=1.251734153196469e-07), sim353: SpearmanrResult(correla

...

epoch: 1, iter: 62000, loss: 30.466468811035156
epoch: 1, iteration: 62000, simlex-999: SpearmanrResult(correlation=0.1572838048897373, pvalue=1.0096640956970655e-06), men: SpearmanrResult(correlation=0.16137007881415613, pvalue=1.405264112416452e-16), sim353: SpearmanrResult(correlation=0.25097676215992787, pvalue=5.8801359002641125e-06), nearest to monster: ['monster', 'robot', 'pen', 'ghost', 'giant', 'cow', 'mine', 'storyline', 'bird', 'blade']

epoch: 1, iter: 62100, loss: 30.18429946899414
epoch: 1, iter: 62200, loss: 30.559833526611328
epoch: 1, iter: 62300, loss: 30.80440902709961
epoch: 1, iter: 62400, loss: 30.450206756591797
epoch: 1, iter: 62500, loss: 30.552818298339844
epoch: 1, iter: 62600, loss: 30.82094383239746
epoch: 1, iter: 62700, loss: 30.254344940185547
epoch: 1, iter: 62800, loss: 30.72846221923828
epoch: 1, iter: 62900, loss: 30.654434204101562
epoch: 1, iter: 63000, loss: 30.073328018188477
epoch: 1, iter: 63100, loss: 30.521087646484375
epoch: 1, iter: 63200,

epoch: 1, iteration: 74000, simlex-999: SpearmanrResult(correlation=0.1617221261012147, pvalue=4.916430156015046e-07), men: SpearmanrResult(correlation=0.16269528376899162, pvalue=7.876945371094953e-17), sim353: SpearmanrResult(correlation=0.2568775911038176, pvalue=3.4697706327940896e-06), nearest to monster: ['monster', 'giant', 'clown', 'robot', 'triangle', 'killer', 'horn', 'storyline', 'bird', 'pen']

epoch: 1, iter: 74100, loss: 30.254886627197266
epoch: 1, iter: 74200, loss: 29.888710021972656
epoch: 1, iter: 74300, loss: 30.417236328125
epoch: 1, iter: 74400, loss: 30.457595825195312
epoch: 1, iter: 74500, loss: 31.00020980834961
epoch: 1, iter: 74600, loss: 30.30846405029297
epoch: 1, iter: 74700, loss: 30.387718200683594
epoch: 1, iter: 74800, loss: 30.376087188720703
epoch: 1, iter: 74900, loss: 30.061664581298828
epoch: 1, iter: 75000, loss: 30.370288848876953
epoch: 1, iter: 75100, loss: 30.63956642150879
epoch: 1, iter: 75200, loss: 30.442768096923828
epoch: 1, iter: 7530

epoch: 1, iter: 86100, loss: 30.831403732299805
epoch: 1, iter: 86200, loss: 30.484277725219727
epoch: 1, iter: 86300, loss: 30.60747718811035
epoch: 1, iter: 86400, loss: 30.155363082885742
epoch: 1, iter: 86500, loss: 30.28110122680664
epoch: 1, iter: 86600, loss: 30.374900817871094
epoch: 1, iter: 86700, loss: 30.804969787597656
epoch: 1, iter: 86800, loss: 30.20755958557129
epoch: 1, iter: 86900, loss: 30.167919158935547
epoch: 1, iter: 87000, loss: 30.547744750976562
epoch: 1, iter: 87100, loss: 30.687185287475586
epoch: 1, iter: 87200, loss: 30.32683563232422
epoch: 1, iter: 87300, loss: 30.641101837158203
epoch: 1, iter: 87400, loss: 30.987831115722656
epoch: 1, iter: 87500, loss: 30.438377380371094
epoch: 1, iter: 87600, loss: 30.0216007232666
epoch: 1, iter: 87700, loss: 30.663925170898438
epoch: 1, iter: 87800, loss: 30.71135711669922
epoch: 1, iter: 87900, loss: 30.71870994567871
epoch: 1, iter: 88000, loss: 30.205699920654297
epoch: 1, iteration: 88000, simlex-999: Spearman

epoch: 1, iter: 98200, loss: 30.314851760864258
epoch: 1, iter: 98300, loss: 30.17748260498047
epoch: 1, iter: 98400, loss: 30.060449600219727
epoch: 1, iter: 98500, loss: 30.29900550842285
epoch: 1, iter: 98600, loss: 30.583925247192383
epoch: 1, iter: 98700, loss: 30.511886596679688
epoch: 1, iter: 98800, loss: 29.978679656982422
epoch: 1, iter: 98900, loss: 30.08024787902832
epoch: 1, iter: 99000, loss: 29.74579620361328
epoch: 1, iter: 99100, loss: 30.44879722595215
epoch: 1, iter: 99200, loss: 30.379261016845703
epoch: 1, iter: 99300, loss: 29.564411163330078
epoch: 1, iter: 99400, loss: 30.413551330566406
epoch: 1, iter: 99500, loss: 29.98810386657715
epoch: 1, iter: 99600, loss: 30.30841827392578
epoch: 1, iter: 99700, loss: 30.51578140258789
epoch: 1, iter: 99800, loss: 30.445234298706055
epoch: 1, iter: 99900, loss: 30.237821578979492
epoch: 1, iter: 100000, loss: 30.199050903320312
epoch: 1, iteration: 100000, simlex-999: SpearmanrResult(correlation=0.16715232584964468, pvalu

epoch: 1, iter: 110100, loss: 30.272464752197266
epoch: 1, iter: 110200, loss: 30.38793182373047
epoch: 1, iter: 110300, loss: 30.590267181396484
epoch: 1, iter: 110400, loss: 30.97867202758789
epoch: 1, iter: 110500, loss: 30.195693969726562
epoch: 1, iter: 110600, loss: 30.050588607788086
epoch: 1, iter: 110700, loss: 30.010971069335938
epoch: 1, iter: 110800, loss: 30.200347900390625
epoch: 1, iter: 110900, loss: 30.716394424438477
epoch: 1, iter: 111000, loss: 30.02122688293457
epoch: 1, iter: 111100, loss: 30.24693489074707
epoch: 1, iter: 111200, loss: 30.085987091064453
epoch: 1, iter: 111300, loss: 30.499698638916016
epoch: 1, iter: 111400, loss: 30.532825469970703
epoch: 1, iter: 111500, loss: 29.860715866088867
epoch: 1, iter: 111600, loss: 30.18459701538086
epoch: 1, iter: 111700, loss: 30.063079833984375
epoch: 1, iter: 111800, loss: 30.4438533782959
epoch: 1, iter: 111900, loss: 29.979290008544922
epoch: 1, iter: 112000, loss: 29.959312438964844
epoch: 1, iteration: 112000

In [11]:
model.load_state_dict(torch.load("embedding-{}.th".format(EMBEDDING_SIZE)))

## 在 MEN 和 Simplex-999 数据集上做评估

In [12]:
embedding_weights = model.input_embeddings()
print("simlex-999", evaluate("simlex-999.txt", embedding_weights))
print("men", evaluate("men.txt", embedding_weights))
print("wordsim353", evaluate("wordsim353.csv", embedding_weights))

simlex-999 SpearmanrResult(correlation=0.17251697429101504, pvalue=7.863946056740345e-08)
men SpearmanrResult(correlation=0.1778096817088841, pvalue=7.565661657312768e-20)
wordsim353 SpearmanrResult(correlation=0.27153702278146635, pvalue=8.842165885381714e-07)


## 寻找nearest neighbors

In [13]:
for word in ["good", "fresh", "monster", "green", "like", "america", "chicago", "work", "computer", "language"]:
    print(word, find_nearest(word))

good ['good', 'bad', 'perfect', 'hard', 'questions', 'alone', 'money', 'false', 'truth', 'experience']
fresh ['fresh', 'grain', 'waste', 'cooling', 'lighter', 'dense', 'mild', 'sized', 'warm', 'steel']
monster ['monster', 'giant', 'robot', 'hammer', 'clown', 'bull', 'demon', 'triangle', 'storyline', 'slogan']
green ['green', 'blue', 'yellow', 'white', 'cross', 'orange', 'black', 'red', 'mountain', 'gold']
like ['like', 'unlike', 'etc', 'whereas', 'animals', 'soft', 'amongst', 'similarly', 'bear', 'drink']
america ['america', 'africa', 'korea', 'india', 'australia', 'turkey', 'pakistan', 'mexico', 'argentina', 'carolina']
chicago ['chicago', 'boston', 'illinois', 'texas', 'london', 'indiana', 'massachusetts', 'florida', 'berkeley', 'michigan']
work ['work', 'writing', 'job', 'marx', 'solo', 'label', 'recording', 'nietzsche', 'appearance', 'stage']
computer ['computer', 'digital', 'electronic', 'audio', 'video', 'graphics', 'hardware', 'software', 'computers', 'program']
language ['langu

## 单词之间的关系

In [14]:
man_idx = word_to_idx["man"] 
king_idx = word_to_idx["king"] 
woman_idx = word_to_idx["woman"]
embedding = embedding_weights[woman_idx] - embedding_weights[man_idx] + embedding_weights[king_idx]
cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
for i in cos_dis.argsort()[:20]:
    print(idx_to_word[i])

king
henry
charles
pope
queen
iii
prince
elizabeth
alexander
constantine
edward
son
iv
louis
emperor
mary
james
joseph
frederick
francis
